In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy import sparse

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.linear_model import LinearRegression
from sklearn import svm

from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.linear_model import Ridge




## Loading training Data

In [3]:
train = pd.read_csv('D://Project//NLP//dataset//train.csv',  usecols=['target', 'comment_text'])

## Target Variable

In [4]:
y = train['target']
del train['target']

## Train- Test Split

In [5]:
X_train, X_test, y_train, y_test = train_test_split( train, y, test_size=0.33, random_state=0)

# Data Preprocessing

In [6]:
def text_prepare(text): ### The function will take in text and lower case it remove the stopwords, symbols and return it
    
    STOPWORDS = set(stopwords.words('english'))

    text = text.lower()                          ### Write a code which can change the input text to lowercase
    text = re.compile('[/(){}\[\]\|@,;]').sub(' ', text)    ### Write a code which replaces REPLACE_BY_SPACE_RE symbols by space in text
    text = re.compile('[^0-9a-z #+_]').sub('', text)          ### Write a code which deletes symbols which are in BAD_SYMBOLS_RE from text
    pattern = re.compile(r'\b(' + r'|'.join(STOPWORDS) + r')\b\s*') 
    text = pattern.sub('', text)                 ### Write a code which deletes stopwords from text
    text = re.sub(' +', ' ', text)
        
    return text

# Handle missing Comments by unknown Value and Pre-process the data

In [7]:
COMMENT = 'comment_text'
X_train[COMMENT].fillna("unknown", inplace=True)
X_test[COMMENT].fillna("unknown", inplace=True)
X_train[COMMENT] = [text_prepare(COMMENT) for COMMENT in X_train[COMMENT]]
X_test[COMMENT] = [text_prepare(COMMENT) for COMMENT in X_test[COMMENT]]


C:\Users\manis\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


# Create Backup of the train test split

In [9]:
back_up_train = X_train
back_up_test = X_test

## TF_IDF Vectorizer

In [10]:
vect_word = TfidfVectorizer(max_features=30000, lowercase=True, analyzer='word',stop_words= 'english',dtype=np.float32)

In [11]:
tr_vect = vect_word.fit_transform(X_train['comment_text'])
ts_vect = vect_word.transform(X_test['comment_text'])

# Models 

In [12]:
X = sparse.hstack([tr_vect])
x_test = sparse.hstack([ts_vect])

## Linear Regression

In [13]:
lr = LinearRegression()
lr.fit(X,y_train)
#cv_score.append(lr.score)
pred_linear = lr.predict(x_test)

In [14]:
mean_squared_error(y_test, pred_linear)

0.01874516836360147

## Lasso Regression

In [15]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(X,y_train)
pred_lasso = clf.predict(x_test)

In [16]:
mean_squared_error(y_test, pred_lasso)

0.03904182858315246

## Ridge Regression

In [17]:
clf = Ridge(alpha=1.0)
clf.fit(X,y_train)
pred_ridge = clf.predict(x_test)

In [18]:
mean_squared_error(y_test, pred_ridge)

0.018144322740151028

## Support Vector Regression

In [ ]:
clf = svm.SVR()
clf.fit(X, y_train) 
SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)
pred_svm = clf.predict(x_test)



C:\Users\manis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
mean_squared_error(y_test, pred_svm)